# Exploratory Analysis of Import Data

This notebook explores the PIERS Bill of Lading data, obtained from S&P's Global Trade Analytics Suite. See the README.md file for more info on the overall project, data pre-processing, and column definitions. 

In [ ]:
#import libraries
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns


#display settings
pd.set_option('display.max_columns', None)
%matplotlib inline

#enable string cache for polars categoricals
pl.enable_string_cache()

## Basic Summary Stats

In [ ]:
#set paths
imports_path = 'data/clean/imports/'
exports_path = 'data/clean/exports/'

#get schema and col names
imports_schema = pl.read_parquet_schema(source= imports_path+'piers_imports_2005.parquet')
imports_colnames = imports_schema.keys()
exports_schema = pl.read_parquet_schema(source=exports_path+'piers_exports_complete.parquet')
exports_colnames = exports_schema.keys()

#init lazy dataframes
imports_lzdf = pl.scan_parquet(imports_path+'*.parquet', parallel='columns')
exports_lzdf = pl.scan_parquet(exports_path+'piers_exports_complete.parquet', parallel='columns')

#get number of observations
imports_n = imports_lzdf.select(pl.count()).collect().item()
exports_n = exports_lzdf.select(pl.count()).collect().item()
print('The imports dataset has {:,} unique observations and {} columns.'.format(imports_n, len(imports_schema)))
print('The exports dataset has {:,} unique observations and {} columns.'.format(exports_n, len(exports_schema)))

In [ ]:
#view head of imports dataframe 
imports_lzdf.limit(n=3).collect()

In [ ]:
#view head of exports dataframe 
exports_lzdf.limit(n=3).collect()

In [ ]:
#init df and get stats labels column
imports_summarystats_df = imports_lzdf.select(pl.first()).collect().describe().select(pl.first()).to_pandas()
#loop through columns and get descriptive stats
for column in imports_colnames:
    imports_summarystats_df[column] = imports_lzdf.select(pl.col(column)).collect().describe().select(column).to_pandas()
#display
imports_summarystats_df

In [ ]:
#NOTE at the moment the exports dataset may fit in memory, in which case the below code could be accomplished more efficiently
#by executing the following line; however, the main code below should run even when the dataset does not fit in memory. 
#exports_lzdf.collect().describe()

#init df and get stats labels column
exports_summarystats_df = exports_lzdf.select(pl.first()).collect().describe().select(pl.first()).to_pandas()
#loop through columns and get descriptive stats
for column in exports_colnames:
    exports_summarystats_df[column] = exports_lzdf.select(pl.col(column)).collect().describe().select(column).to_pandas()
#display
exports_summarystats_df

## Duplicate BOLs 

In [ ]:
#count unique bol_scac IDs
import_bols_unique_n = imports_lzdf.select(pl.col('bol_id')).unique().select(pl.count()).collect().item()
export_bols_unique_n = exports_lzdf.select(pl.col('bol_id')).unique().select(pl.count()).collect().item()

print('{:,} out of {:,} rows in the imports dataset contain duplicated BoLs.'.format(imports_n-import_bols_unique_n, imports_n))
print('{:,} out of {:,} rows in the imports dataset contain duplicated BoLs.'.format(exports_n-export_bols_unique_n, exports_n))

Possible reasons:
- bol includes items on more than one ship


194.8k import bols share a bol-scac ID but have different vessel IDs. 
664.1k export bols share a bol-scac ID but have different vessel IDs.

## Entries with zero weight:

In [ ]:
pldf[pldf.weight == 0].describe()

In [ ]:
pldf.describe()

## Value and Volumes by Year:

In [ ]:
#get year col
pldf['year'] = pldf.date_arrival.dt.to_period('Y')
#group value and volume by year
activityacrosstime_df = pldf[['year', 'teus', 'value_est']].groupby('year').sum()
#plot
sns.barplot(data=activityacrosstime_df, x='year', y='value_est');
plt.title('Total Value of Imports Over Time')
plt.xticks(rotation=45);

In [ ]:
#plot
sns.barplot(data=activityacrosstime_df, x='year', y='teus');
plt.title('Total Volume (TEUs) of Imports Over Time')
plt.xticks(rotation=45);

I guess value and volume records weren't kept before ~2015 ?!?

In [ ]:
#group value and volume by year
until2012_df = pldf[pldf['year']< pd.Period(2013)]
until2012_df = until2012_df[['year', 'teus', 'value_est']].groupby('year').sum()
#plot
sns.barplot(data=until2012_df, x='year', y='teus');
plt.title('Total Volume (TEUs) of Imports Over Time')
plt.xticks(rotation=45);

Must be missing data here?

In [ ]:
#get year col
pldf['year'] = pldf.date_arrival.dt.to_period('Y')
#group value and volume by year
activityacrosstime_df = pldf[['year', 'container_piece_count']].groupby('year').sum()
#plot
sns.barplot(data=activityacrosstime_df, x='year', y='container_piece_count');
plt.title('Total Volume (Container Piece Count) of Imports Over Time')
plt.xticks(rotation=45);

In [ ]:
pldf.head()

## Carriers Over Time

In [ ]:
carriersovertime_df = pldf[['year', 'carrier_scac']].groupby('year').nunique()

In [ ]:
sns.barplot(data=carriersovertime_df, x='year', y='carrier_scac');
plt.title('Number of Unique Carriers Over Time');
plt.xticks(rotation=45);

Was there really a spike in carriers in 2010 and 2012? Or does this indicate changes in the way SCAC codes are assigned?

### Market share of the 50 largest carriers by estimated value 

In [ ]:
#get largest carriers
carriers_df = pldf[pldf.year > pd.Period(2014)]
carriers_df = carriers_df[['year', 'carrier_scac', 'value_est']].groupby(['year', 'carrier_scac']).sum()

In [ ]:
carriers_df.columns = ['value_usd']
carriers_df.sort_values('value_usd', ascending=False, inplace=True)
carriers_df.sort_values('year', inplace=True)

In [ ]:
carriers_df.reset_index(inplace=True)
carriers_df.head()